In [6]:
# import external libraries
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service as ChromeService 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm
from typing import *
import random
from itertools import islice
import pandas as pd
from collections import defaultdict
import time
from urllib.parse import urlparse

import warnings
warnings.filterwarnings('ignore')


In [7]:


# ************************************************************************* Selenium configuration ****************************************************************
service = ChromeService()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)
# driver.implicitly_wait(10)
wait = WebDriverWait(driver, 5)
driver.maximize_window()



# ************************************************************************* Data ******************************************************************************


# Url 
url_base = 'https://www.linkedin.com/search/results/companies'


data_dict=dict(
    company_name=[],
    linkedin_url=[]
)


# ************************************************************************* Functions ******************************************************************************
def login(driver) -> None:
# Login
    sign_in_link=driver.find_element(By.XPATH,'/html/body/div[1]/main/div/p/a')
    sign_in_link.click()
    time.sleep(3*random.random())

    email_input=driver.find_element(By.XPATH,'//*[@id="username"]')
    email_input.send_keys('ariel.hernan.limes@pwc.com')
    time.sleep(1*random.random())

    password_input=driver.find_element(By.XPATH,'//*[@id="password"]')
    password_input.send_keys('MiVida1855')
    time.sleep(1*random.random())

    signIn_Button=driver.find_element(By.XPATH,'//*[@id="organic-div"]/form/div[3]/button')
    signIn_Button.click()
    time.sleep(3*random.random())
    

def validate_profile(company):
    
    global data_dict
    
    # Url 
    url_base = 'https://www.linkedin.com/search/results/companies'
    # Definint the driver
    driver.get(url_base)
    
    company_search = driver.find_element(By.XPATH,'//input[@class="search-global-typeahead__input"]')
    company_search.clear()
    company_search.send_keys(company)
    company_search.send_keys(Keys.ENTER)
    time.sleep(2)

    try:
        company_container=driver.find_elements(By.XPATH,'//li[@class="reusable-search__result-container"]')
        
        time.sleep(1)
        if len(company_container) > 0:
        
            time.sleep(1)
            company_container[0].click()
            time.sleep(3)
            link_profile=driver.current_url
           
            
            data_dict['company_name'].append(company)
            data_dict['linkedin_url'].append(link_profile)
        else:
            data_dict['company_name'].append(None)
            data_dict['linkedin_url'].append(None)
    except:

        data_dict['company_name'].append(None)
        data_dict['linkedin_url'].append(None)


def explore_tags(company:str, url_company:str, driver)->None:
    
    global data_dict
    
    # Url 
    url_base = 'https://www.linkedin.com/search/results/companies'
    # Definint the driver
    driver.get(url_base)

    company_search = driver.find_element(By.XPATH,'//input[@class="search-global-typeahead__input"]')
    company_search.clear()
    company_search.send_keys(company)
    company_search.send_keys(Keys.ENTER)
    time.sleep(2)    
       
    try: 
        # container with all the companies with the same name    
        container_elements=wait.until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="reusable-search__result-container"]')))
        # star reviewing the companies
        for i in range(len(container_elements)):
            try:
                container_elements=wait.until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="reusable-search__result-container"]')))
                container_elements[i].click()
                time.sleep(1)
                about_link=wait.until(EC.presence_of_element_located((By.XPATH, '//ul[@class="org-page-navigation__items "]/li/a[contains(@href,"about")]')))
                about_link.click()
                
                try:
                    company_url_element=wait.until(EC.presence_of_element_located((By.XPATH, '//dd[@class="mb4 t-black--light text-body-medium"]/a/span')))
                    company_url = company_url_element.text
                except:
                    pass
                
                url_domain = urlparse(company_url).netloc
                target_domain = urlparse(url_company).netloc
                
                if url_domain == target_domain:
                    data_dict['company_name'].append(company)
                    data_dict['linkedin_url'].append(driver.current_url)
                    break

                driver.back()
                time.sleep(2)
                driver.back()
                
            except StaleElementReferenceException:
                
                
                container_elements=wait.until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="reusable-search__result-container"]')))
                container_elements[i].click()
                time.sleep(1)
                about_link=wait.until(EC.presence_of_element_located((By.XPATH, '//ul[@class="org-page-navigation__items "]/li/a[contains(@href,"about")]')))
                about_link.click()

                try:
                    company_url_element=wait.until(EC.presence_of_element_located((By.XPATH, '//dd[@class="mb4 t-black--light text-body-medium"]/a/span')))
                    company_url = company_url_element.text
                except:
                    pass
                
                url_domain = urlparse(company_url).netloc
                target_domain = urlparse(url_company).netloc
                
                if url_domain == target_domain:
                    data_dict['company_name'].append(company)
                    data_dict['linkedin_url'].append(driver.current_url)
                    break

                driver.back()
                time.sleep(2)
                driver.back()
    except:
        pass


In [13]:

# *********************************************************** Reading Data to Explore ****************************************************************************
# df_summary=pd.read_excel(r'../../Output/Results/summary.xlsx') 
# df_base=pd.read_parquet(r'C:\Users\alimes001\Documents\DAT EU\REPOS\DA-EU\DA-Codes repository\src\Dev\WebCrawler\Output\Results\combined_file_html_to_text.parquet')
# df_output=df_base.drop_duplicates(subset=['url_base'])
# df_output.drop(columns=['text','url'], inplace=True)

# df_merge=pd.merge(df_summary, df_output, on='url_base', how='left')

# df_merge['url_base_right']=df_merge['url_base'].apply(lambda x: x.replace('https://','https://www.').replace('http://','http://www.'))
# df_merge.to_excel(r'../../Output/Results/Summ_Linkedin.xlsx', index=False)

df_base=pd.read_parquet(r'C:\Users\alimes001\Documents\DAT EU\REPOS\DA-EU\DA-Codes repository\src\Dev\WebCrawler\Output\Results\combined_file_html_to_text.parquet')
df_output=df_base.drop_duplicates(subset=['url_base'])
df_output.drop(columns=['text','url'], inplace=True)
df_output=df_output[~df_output['linkedin_profile'].isna()]


In [15]:

# ************************************************************************ Execution **************************************************************************


with tqdm(total=len(df_output)) as pbar:
    for index , row in df_output.iterrows():
        company=row['Project']
        url_company=row['url_base']
        explore_tags(company, url_company, driver)
        time.sleep(2)
        pbar.update(1)
        
        


 90%|████████▉ | 562/627 [2:39:23<18:26, 17.02s/it]  


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@class="search-global-typeahead__input"]"}
  (Session info: chrome=119.0.6045.200); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7FE8C82B2+55298]
	(No symbol) [0x00007FF7FE835E02]
	(No symbol) [0x00007FF7FE6F05AB]
	(No symbol) [0x00007FF7FE73175C]
	(No symbol) [0x00007FF7FE7318DC]
	(No symbol) [0x00007FF7FE76CBC7]
	(No symbol) [0x00007FF7FE7520EF]
	(No symbol) [0x00007FF7FE76AAA4]
	(No symbol) [0x00007FF7FE751E83]
	(No symbol) [0x00007FF7FE72670A]
	(No symbol) [0x00007FF7FE727964]
	GetHandleVerifier [0x00007FF7FEC40AAB+3694587]
	GetHandleVerifier [0x00007FF7FEC9728E+4048862]
	GetHandleVerifier [0x00007FF7FEC8F173+4015811]
	GetHandleVerifier [0x00007FF7FE9647D6+695590]
	(No symbol) [0x00007FF7FE840CE8]
	(No symbol) [0x00007FF7FE83CF34]


In [27]:

# ************************************************ Saving the results ****************************************************************************************** 
df_result=pd.DataFrame(data_dict)
df_full=pd.merge(df_output, df_result, left_on='Project', right_on='company_name', how='left')


0

In [29]:
df_output.to_excel(r'../../Output/Results_2/Summ_Linkedin.xlsx', index=False)

In [ ]:

df_full['linkedin_profile']=df_full.apply(lambda row: row['linkedin_url'] if pd.isna(row['linkedin_profile']) else row['linkedin_profile'],axis=1)
df_full.drop(columns=['company_name','linkedin_url'], inplace=True)

df_glass=pd.read_excel(r"C:\Users\alimes001\Downloads\sum__linkedin_output.xlsx")

df_integrated=pd.merge(df_full, df_glass, left_on='Company',right_on='company_source', how='left')
df_integrated.drop(columns=['company_url_source','url_base_x','url_base_y','company_source'], inplace=True)

In [107]:
df_integrated.duplicated(subset=['Company','linkedin_profile']).sum()

0

In [106]:

df_integrated.to_excel(r'../../Output/Results/Summ-Glassdoor.xlsx', index=False)